In [1]:
!pip install polars --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install duckdb --no-index --find-links=file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg

Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg/polars-0.20.16-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: file:///kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [2]:
import polars as pl
import duckdb as dd
import os
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import tensorflow as tf
# import tensorflow_io as tfio
from tensorflow import keras

E0000 00:00:1730432728.076758      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1101 03:45:28.085776939      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1101 03:45:28.085791885      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1101 03:45:28.085795494      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1101 03:45:28.085797990      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1101 03:45:28.085800518      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

In [3]:
# detect TPUs
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1730432741.698459      13 service.cc:145] XLA service 0x59248edaca30 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730432741.698516      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1730432741.698521      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1730432741.698524      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1730432741.698527      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1730432741.698530      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1730432741.698533      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1730432741.698536      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1730432741.698538      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 96


In [5]:
Test = True

if Test:
    num_jobs = 92
else:
    num_jobs = 4

config_dict = {'train_csv_path': '/kaggle/input/child-mind-institute-problematic-internet-use/train.csv'}
config_dict['test_csv_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/test.csv'
config_dict['train_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/'
config_dict['test_parquet_root_path'] = '/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet/'

In [6]:
train_data = pl.read_csv(config_dict['train_csv_path'], low_memory=True)
# train_data.describe()
train_data_id_list = pl.Series(train_data.select(pl.col('id'))).to_list()
# len(train_data_id_list)

# actigraphy_file_list = []
actigraphy_id_list = []

for dirname, _, filenames in os.walk(config_dict['train_parquet_root_path']):
    for filename in filenames:
        full_parquet_file_path = os.path.join(dirname, filename)
        # actigraphy_file_list.append(full_parquet_file_path)
        actigraphy_id_list.append(full_parquet_file_path.split('/')[-2].replace('id=',''))
        
final_id_list = list(set(actigraphy_id_list).intersection(train_data_id_list))

print(len(final_id_list))
print(len(train_data_id_list))

996
3960


In [7]:
categorical_colms = ['Basic_Demos-Enroll_Season','CGAS-Season','Physical-Season','Fitness_Endurance-Season','FGC-Season'\
                    ,'BIA-Season','PCIAT-Season','PreInt_EduHx-Season','PAQ_A-Season','PAQ_C-Season','SDS-Season']

for cat_col in categorical_colms:
    cat_vals_list = pl.Series(train_data.select(pl.col(cat_col))).to_list()
    encoded_vals_list = le.fit_transform(cat_vals_list)
    encoded_col = "encoded_{0}".format(cat_col)
    train_data = train_data.with_columns(pl.Series(name=encoded_col, values=encoded_vals_list))
    train_data = train_data.drop([cat_col])

In [8]:
# Calculate the approximate size of each smaller list
chunk_size = len(train_data_id_list) // num_jobs
remainder = len(train_data_id_list) % num_jobs

# Create the num_jobs lists
result = []
start = 0

for i in range(num_jobs):
    # Add an extra element to the first few lists to account for remainder
    end = start + chunk_size + (1 if i < remainder else 0)
    result.append(train_data_id_list[start:end])
    start = end

# Check the result
for i, sublist in enumerate(result):
    print(f"List {i + 1} (length {len(sublist)}): {sublist}")

List 1 (length 44): ['00008ff9', '000fd460', '00105258', '00115b9f', '0016bb22', '001f3379', '0038ba98', '0068a485', '0069fbed', '0083e397', '0087dd65', '00abe655', '00ae59c9', '00af6387', '00bd4359', '00c0cd71', '00d56d4b', '00d9913d', '00e6167c', '00ebc35d', '00f332d1', '00f574e9', '01085eb3', '01182ce3', '011e71c3', '012cadd8', '012e3869', '014c303e', '015ba5b1', '0194c251', '019c9266', '01cae1ef', '01d24f60', '01dc6dcd', '01e9a5c0', '01f83da1', '02073150', '02132c62', '0219f64a', '021b2020', '0232fd98', '02347f13', '023e1b6a', '023eba7b']
List 2 (length 44): ['026102c4', '02648a1c', '026fcbc3', '0274e7f3', '029a19c9', '02a01f18', '02c4cf7f', '02cebf33', '02cf7384', '02dcf62a', '02e56a7c', '0347462d', '034f923b', '0357a10e', '035c96dd', '0380e709', '03833cec', '0391e587', '03a5c2d6', '03a9019b', '03bf5026', '03e83fef', '03f1c1c8', '040d103f', '0417c91e', '04189592', '04370442', '0441d6af', '045a0a94', '0470efa4', '0471236c', '0499c3e8', '04a1633e', '04ad419d', '04aeed24', '04af8fdd'

In [9]:
def get_hours(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[0])

def get_mins(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[1])
    
def get_secs(time_of_day):
    ts = int(time_of_day)
    ts /= 1000
    date_var = datetime.utcfromtimestamp(ts).strftime('%H:%M:%S')
    return int(date_var.split(':')[2])

In [16]:
parquet_file_name = "id={0}/".format(final_id_list[2])
first_parquet_file = config_dict['train_parquet_root_path']+parquet_file_name

directory = Path(first_parquet_file)

# Check if any Parquet file exists in the directory
parquet_files = list(directory.glob("*.parquet"))

if parquet_files:
    print("Parquet files found:", parquet_files)
else:
    print("No Parquet files found in the directory.")

Parquet files found: [PosixPath('/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id=f285b43c/part-0.parquet')]


In [22]:
def transform_parquet_file(parquet_file_path, file_id):

    directory = Path(parquet_file_path)

    # Check if any Parquet file exists in the directory
    parquet_files = list(directory.glob("*.parquet"))

    if parquet_files:
        parquet_df = pl.read_parquet(parquet_file_path+"*.parquet")
        parquet_df = parquet_df.drop(['id'])
        parquet_df = parquet_df.with_columns([
            pl.lit(file_id).alias("id"),
            pl.col("time_of_day").map_elements(get_hours, return_dtype=pl.Int32).alias("hour_of_day")
        ])

        parquet_df = parquet_df.drop(['time_of_day'])
        parquet_df = parquet_df.group_by(["id","weekday","hour_of_day","relative_date_PCIAT"])\
        .agg([pl.mean("enmo").name.suffix("_avg"), pl.mean("anglez").name.suffix("_avg"), pl.mean("light").name.suffix("_avg")\
              , pl.mean("battery_voltage").name.suffix("_avg")])
    else:
        columns = [("id", pl.String), ("weekday", pl.Int8), ("hour_of_day", pl.Int32), ("relative_date_PCIAT", pl.Float32)\
                  , ("enmo_avg", pl.Float32), ("anglez_avg", pl.Float32), ("light_avg", pl.Float32)\
                   , ("battery_voltage_avg", pl.Float32)]

        parquet_df = pl.DataFrame(schema=columns)
    
    return parquet_df

In [23]:
def unify_files(list_of_ids):
    df_initializing_id = list_of_ids.pop()
    parquet_file_name = "id={0}/".format(df_initializing_id)
    first_parquet_file = config_dict['train_parquet_root_path']+parquet_file_name
    first_parquet_df = transform_parquet_file(first_parquet_file, df_initializing_id)
    
    train_data_of_id = train_data.filter(pl.col('id')==df_initializing_id)
    first_joined_features_df = train_data_of_id.join(first_parquet_df, on="id", how="left")
    
    for id in tqdm(list_of_ids):
        parquet_file_name = "id={0}/".format(id)
        parquet_file_path = config_dict['train_parquet_root_path']+parquet_file_name
        parquet_df = transform_parquet_file(parquet_file_path, id)
        
        train_data_of_id = train_data.filter(pl.col('id')==id)
        joined_features = train_data_of_id.join(parquet_df, on="id", how="left")
        
        try:
            first_joined_features_df = pl.concat([first_joined_features_df, joined_features])
        except:
            print(id)
    return first_joined_features_df

In [ ]:
parts_of_parquet = (Parallel(n_jobs=num_jobs)(delayed(unify_files)(id_list) for id_list in result))

 29%|██▊       | 12/42 [00:18<00:54,  1.81s/it]

In [28]:
len(parts_of_parquet)

92

In [33]:
for i in range(len(parts_of_parquet)):
    print(parts_of_parquet[i].shape)

(44, 89)
(44, 89)
(44, 89)
(44, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)
(43, 89)


In [32]:
test_df = parts_of_parquet[0]
dd.sql("select * from test_df limit 100").pl()

id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,…,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii,encoded_Basic_Demos-Enroll_Season,encoded_CGAS-Season,encoded_Physical-Season,encoded_Fitness_Endurance-Season,encoded_FGC-Season,encoded_BIA-Season,encoded_PCIAT-Season,encoded_PreInt_EduHx-Season,encoded_PAQ_A-Season,encoded_PAQ_C-Season,encoded_SDS-Season,weekday,hour_of_day,relative_date_PCIAT,enmo_avg,anglez_avg,light_avg,battery_voltage_avg
str,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i8,i32,f32,f32,f32,f32,f32
"""023eba7b""",16,0,null,21.937701,66.5,138.0,31.0,70,99,124,null,null,null,25,1,30.84,2,34.02,2,20,1,0.0,0,4.0,0,12.0,1,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3,null,3,4,3,4,1,4,4,3,3,4,4,null,null,null,null,null,null,null
"""00008ff9""",5,0,51,16.877316,46.0,50.8,null,null,null,null,null,null,null,0,0,null,null,null,null,0,0,7.0,0,6.0,0,6.0,1,2,2.66855,16.8792,932.498,1492.0,8.25598,41.5862,13.8177,3.06143,…,0,4,0,0,4,0,4,4,4,4,4,4,2,4,55,null,null,3,2,0,3,0,4,0,0,0,0,4,4,4,null,null,null,null,null,null,null
"""000fd460""",9,0,null,14.03559,48.0,46.0,22.0,75,70,122,null,null,null,3,0,null,null,null,null,5,0,11.0,1,11.0,1,3.0,0,2,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,64,0,0,2,4,0,4,0,3,0,2,4,0,0,null,null,null,null,null,null,null
"""00105258""",10,1,71,16.648696,56.5,75.6,null,65,94,117,5,7,33,20,1,10.2,1,14.7,2,7,1,10.0,1,10.0,1,5.0,0,null,null,null,null,null,null,null,null,null,…,1,2,1,1,1,0,1,1,1,0,2,2,1,1,28,38,54,2,0,2,0,0,0,0,4,0,2,4,2,0,null,null,null,null,null,null,null
"""00115b9f""",9,0,71,18.292347,56.0,81.6,null,60,97,117,6,9,37,18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,3,3.84191,18.2943,1131.43,1923.44,15.5925,62.7757,14.074,4.22033,…,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,31,45,0,1,3,0,2,2,2,2,2,3,4,3,2,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""0219f64a""",6,0,68,16.377681,48.5,54.8,24.0,67,81,111,null,null,null,0,0,null,null,null,null,4,1,12.0,1,12.0,1,9.0,1,3,2.89258,16.3795,968.274,1646.07,9.24597,45.3969,13.5689,2.81055,…,0,0,0,2,4,1,2,3,3,3,3,3,2,1,51,69,94,1,2,3,2,1,4,1,2,1,3,4,4,1,null,null,null,null,null,null,null
"""021b2020""",9,1,null,20.631322,53.5,84.0,28.0,64,81,120,null,null,null,9,0,null,null,null,null,3,0,13.0,1,13.0,1,12.0,1,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0,null,3,4,3,4,1,4,4,3,4,3,4,null,null,null,null,null,null,null
"""0232fd98""",14,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,49,68,nu